#### Conceptos de este cuaderno :

## Con que trabaja spark?

##### DataSet(Datos para explotar, normalmente en formato CSV, JSON o similares). Los 'DataFrames'(No distribuido, mutable y no tolerante a fallos) y los 'RDD(Resilient Distributed Dataset)'(Distribuidos, inmutable y tolerante a fallos).

#### En estos primeras celdas trabajaremos con Apache Spark en modo 'Standalone', es decir, que tendremos todo corriendo en una sola máquina.

#### Ips de spark
http://localhost:4040/jobs/  jobs
http://127.0.0.1:8080/ Dashboard

In [3]:
# Que version de Pyspark estamos usando? 

import pyspark
print(pyspark.__version__)

3.5.0


In [4]:
# Que version de Python estamos usando? 
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 08:38:23) 
[Clang 14.0.6 ]


### Optimizacion de recursos

##### PySpark utiliza memoria tanto para el almacenamiento en caché como para el procesamiento. Puedes configurar la cantidad de memoria que PySpark debe utilizar para cada uno de estos propósitos

In [5]:
#Configuración de memoria
from pyspark import SparkConf
conf = SparkConf().setAppName("EcommerceCosmeticShop").set("spark.driver.memory", "4g").set("spark.executor.memory", "4g")


#### Paralelismo y número de ejecutores: Otra configuración importante es el número de ejecutores y el nivel de paralelismo. Esto afecta cómo se divide la carga de trabajo. Puedes configurar la cantidad de ejecutores y los hilos de cada executor.

In [6]:
#Configuracion de Paralelismo y número de ejecutores 
conf.set("spark.executor.instances", "4").set("spark.executor.cores", "2")

In [7]:
# crear un dataframe de cliente declarando el esquema y pasando valores
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.sql import SparkSession

In [8]:
# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("EcommerceCosmeticShop") \
    .master("spark://MBPdeLucas880.fibertel.com.ar:7077") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/03 18:49:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# terminal 
#Arrancamos el master en modo standalone(Arranca WebUI en 127.0.0.1:8080)
$SPARK_HOME/sbin/start-master.sh

 Arrancamos el primer esclavo
$SPARK_HOME/sbin/start-slave.sh spark://

# PYSPARK - DATAFRAME

In [9]:
# Leemos datos del DataSet y los escribimos en un DataFrame
df = spark.read.options(header='True', inferSchema='True').csv(['2019-Dec.csv', '2019-Nov.csv', '2019-Oct.csv', '2020-Feb.csv', '2020-Jan.csv'])



In [ ]:
# Escribimos el DataFrame en disco en la carpeta donde se encuentra este Jupyter Notebook
df.write.mode('overwrite').csv("ecommerce-cosmetic-shop")

##### Detener la sesión de Spark cuando hayas terminado
spark.stop()


Comprobar que el proceso ha generado ficheros CSVs. El motivo es que Apache Spark particiona los datos para procesar en paralelo y, por lo tanto, crea un fichero por partición(También ha creado un fichero '_SUCCESS' para indicar el momento en que había acabado de exportar).

In [8]:
# Contar número de registros de un DataFrame
df.count()

20692840

In [9]:
# Escribir el Schema de los datos del Data Frame
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [10]:
# Mostrar o contar los diferentes valores de una columna
df.select('event_type').distinct().show()

+----------------+
|      event_type|
+----------------+
|        purchase|
|            view|
|            cart|
|remove_from_cart|
+----------------+



In [11]:
# Mostrar o contar los diferentes valores de una columna
df.select('product_id').distinct().count()

54571

In [25]:
df.select('brand').distinct().show()

+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|     philips|
|invisibobble|
|       riche|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
|protokeratin|
|       essie|
+------------+
only showing top 20 rows



In [13]:
# Obtener el primer 'product_id' del registro con 'event_type=cart'
df.select(['product_id']).filter("event_type='cart'").first()

Row(product_id=4958)

In [16]:
# Obtener los productos que se han comprado conjuntamente con el anterior registro
sesions=df.select(['user_session']).filter("event_type='cart' AND product_id=5844305").distinct().first()

In [15]:
products=df.select(['product_id']).filter("event_type='cart' AND product_id<>5844305").filter( df["user_session"].isin(sesions["user_session"]))
products.select('product_id').show()

+----------+
|product_id|
+----------+
|   5820774|
|   5820776|
|   5820777|
|   5870111|
|   5844303|
|   5820721|
|   5698887|
|   5698879|
+----------+



### SQL API - opcion b

##### La principal desventaja a la hora de utilizar SQL en vez de la funcionalidad de spark es que solo podremos trabajar con dataframes

In [18]:
from pyspark.sql import SparkSession

# Crear un objeto SparkSession
sparksql = SparkSession.builder.appName("EcommerceCosmeticShop").getOrCreate()

# Lista de archivos CSV
file_paths = ['2019-Dec.csv', '2019-Nov.csv', '2019-Oct.csv', '2020-Feb.csv', '2020-Jan.csv']

# Cargar los datos desde los archivos CSV
df = sparksql.read.csv(file_paths, header=True, inferSchema=True)

# Registrar la tabla en Spark
df.createOrReplaceTempView("EcommerceCosmeticShop")



In [19]:
# Ahora podemos ejecutar consultas SQL en la tabla registrada
result = sparksql.sql("SELECT * FROM EcommerceCosmeticShop")
result.show()


Py4JError: An error occurred while calling o25.sql. Trace:
py4j.Py4JException: Method sql([class java.lang.String, class [Ljava.lang.Object;]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:832)



# RDDs

In [9]:
df = spark.read.options(header='True', inferSchema='True').csv(['2019-Dec.csv', '2019-Nov.csv', '2019-Oct.csv', '2020-Feb.csv', '2020-Jan.csv'])
df.count()
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [10]:
df.select('brand').distinct().show()

+------------+
|       brand|
+------------+
|     beautix|
|     farmona|
|  dr.gloderm|
|   profhenna|
|     philips|
|invisibobble|
|       riche|
|        oniq|
|    lebelage|
|     vilenta|
|       fancy|
|      jaguar|
|      tertio|
|    siberina|
|   koreatida|
|         jas|
|rocknailstar|
|   depilflax|
|protokeratin|
|       essie|
+------------+
only showing top 20 rows



In [13]:
# Creamos una funcion donde obtendremos los registros de los productID de la brand Richie

In [14]:
def myFunc(s):
    result = []
    if s["brand"] == "riche" and s["event_type"] == "cart":
        result.append((s["product_id"], 1))
    return result

#### Vamos a utilizar MapReduce y lambdas (tambien es posible pasar funciones) pero con python y esto es un limitante porque a diferencia de Scala por que solo podremos colocar una sola instruccion.

In [15]:
lines = df.rdd.flatMap(myFunc).reduceByKey(lambda a, b: a + b)

In [17]:
for element in lines.collect():

IndentationError: expected an indented block (3120165381.py, line 1)

In [18]:
print( lines.take(20) )
print( '\n'.lines.take(20) )
for el in lines.take(20): print(el)

23/11/03 18:55:55 WARN TaskSetManager: Lost task 0.0 in stage 8.0 (TID 60) (192.168.0.164 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
AttributeError: Can't get attribute 'PySparkRuntimeError' on <module 'pyspark.errors.exceptions.base' from '/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/errors/exceptions/base.py'>

	at 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8.0 failed 4 times, most recent failure: Lost task 2.3 in stage 8.0 (TID 73) (192.168.0.164 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
AttributeError: 

In [19]:
lines.toDF().show()
lines.saveAsTextFile("~/documents/projectos_data/ecommerce-shop-Pyspark")

23/11/03 18:56:08 WARN TaskSetManager: Lost task 0.0 in stage 10.0 (TID 77) (192.168.0.164 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
AttributeError: Can't get attribute 'PySparkRuntimeError' on <module 'pyspark.errors.exceptions.base' from '/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/errors/exceptions/base.py'>

	at

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 10.0 failed 4 times, most recent failure: Lost task 3.3 in stage 10.0 (TID 91) (192.168.0.164 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 812, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/Users/lucasmorrone/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
AttributeError